In [266]:
import folium
import json
plt.rc('font', family='AppleGothic')

In [267]:
#importing data

from glob import glob
file_list=glob("data/seoul_crime/*")
crime_total_table=[]

for i, data in enumerate(file_list):
    df=pd.read_csv(data, encoding='euc-kr')
    df['year']=file_list[i].split(".")[0][-7:-3]
    df.dropna(inplace=True)
    crime_total_table.append(df)

crime_total=pd.concat(crime_total_table,ignore_index=True)
crime_raw_data=crime_total[crime_total.columns[:5]].sort_values("year")

In [268]:
#translating kor to eng
crime_raw_data=crime_raw_data.rename(columns={"구분":'class',"죄종":'crime_name',"발생검거":'result',"건수":'count'})

class_name=list(crime_raw_data["class"].unique())
class_list=["Jun-gu","Jongno-gu","Namdaemoon", "Seodaemun-gu", "Hyehwa", "Yongsan-gu", "Seongbuk-gu", "Dongdaemun-gu", "Mapo-gu", "Yeongdeungpo-gu", "Seongdong-gu", "Dongjak-gu", "Gwangjin-gu", "Seobu", "Gangbuk-gu", "Geumcheon-gu", "Jungnang-gu", "Gangnam-gu","Gwanak-gu", "Gangseo-gu", "Gangdong-gu","Jongam", "Guro-gu","Seocho-gu", "Yangcheon-gu", "Songpa-gu","Nowon-gu", "Bangbae", "Eunpyeong-gu","Dobong-gu","Suseo"]
class_dict=dict(zip(class_name,class_list))

crime_name=list(crime_raw_data["crime_name"].unique())
crime_name_dict=dict(zip(crime_name,["merder","burglar","rape","theft","violence"]))

crime_result_dict={"발생":"incident","검거":"arrest"}

#implementing translation
input_list=[class_dict,crime_name_dict,crime_result_dict]
columns=crime_raw_data.columns

for i, dicts in enumerate(input_list):
    crime_raw_data[columns[i]]=crime_raw_data[columns[i]].replace(dicts)

In [269]:
#preprocessing data

# transforming police stations to area 
station_name=["Hyehwa","namdaemoon","Jongam","Seobu","Bangbae","Suseo"]
area_name = ["jongno-gu","Jung-gu","Seongbuk-gu","Eunpyeong-gu","Seocho-gu","Gangnam-gu"]
tmp_dict=dict(zip(station_name,area_name)) 

crime_raw_data["class"]=crime_raw_data["class"].replace(tmp_dict)

#pivot_table
#trenforming dataframe to pivot_table sorted by year and class 
crime_pivot = crime_raw_data.pivot_table(
    crime_raw_data, index=["year","class"], columns=["crime_name","result"], aggfunc=[np.sum])
crime_pivot.columns = crime_pivot.columns.droplevel([0,1])

#simplify class 
tmp = crime_pivot.columns.get_level_values(0)+"_"+crime_pivot.columns.get_level_values(1)
crime_pivot.columns = tmp

#adding arrest rate crime_pivot.loc[year]
arrest=["rape_arrest","burglar_arrest","merder_arrest","theft_arrest","violence_arrest"]
occurs=["rape_incident","burglar_incident","merder_incident","theft_incident","violence_incident"]
new_columns=["rape_arrest_rate","burglar_arrest_rate","merder_arrest_rate","theft_arrest_rate","violence_arrest_rate"]
crime_pivot[new_columns]=crime_pivot[occurs]/crime_pivot[arrest].values*100

#simplify pivot table
crime_pivot= crime_pivot.drop(arrest,axis=1)
crime_pivot[crime_pivot[new_columns]>100]=100

#rename columns 
crime_pivot.rename(columns={
    "rape_incident" : "rape",
    "theft_incident" : "theft",
    "merder_incident" : "merder",
    "burglar_incident" : "burglar",
    "violence_incident" : "violence"},
    inplace = True)
folium_data = crime_pivot

### 1. how often have the viorent crimes occurred in each area?

In [270]:
#normalization by each years
col=["merder","burglar","rape","theft","violence"]
col2= ["merder_arrest_rate","burglar_arrest_rate","rape_arrest_rate","theft_arrest_rate","violence_arrest_rate"]
years=list({i for i  in crime_pivot.index})


for year in years:
    crime_pivot.loc[year][col]=crime_pivot.loc[year][col]/ crime_pivot.loc[year][col].max()
crime_pivot

#adding average crime and arrest rate on area 
crime_pivot["mean_crime"] = np.mean(crime_pivot[col],axis=1)
crime_pivot["mean_arrest"] = np.mean(crime_pivot[col2],axis=1)

### visualization

In [281]:
path='data/03_skorea_municipalities_geo_simple.json'
geo_str=json.load(open(path, encoding="utf-8"))
folium_data["class"]=folium_data.index
folium_data["class"]=folium_data["class"].apply(lambda x:x[1])

In [278]:
import json
geo_path = "/Users/yeonghwanchoi/Documents/git/DS_class/data/03_skorea_municipalities_geo_simple.json"
geo_str = json.load(open(geo_path,encoding="utf-8"))

my_map = folium.Map(location=[37.5502,126.982],zoom_start=11,titles='Stamen Toner')
folium.Choropleth(
    geo_data=new_geo,
    data=folium_data, columns=["class","merder"],
    nan_fill_color="purple", nan_fill_opacity=0.4,
    key_on='feature.properties.name_eng', fill_color="YlGn",
    legend_name=""
).add_to(my_map)

In [279]:
my_map

In [ ]:
for i in range(len(geo_str["features"])):
    new_geo["features"][i]["id"]=geo_str["features.properties.name_eng"][i]["properties"]["name_eng"]